In [9]:
import pandas as pd
def cal_token_trades(data: pd.DataFrame, ADDRESS: str):
    ADDRESS = ADDRESS.lower()
    data['trade_type'] = data.apply(
        lambda x: 'sell' if x['from'] == ADDRESS else 'buy' if x['to'] == ADDRESS else None, axis=1
    )
    
    data = data[data['trade_type'].notnull()]
    buy_summary = data[data['trade_type'] == 'buy'].groupby('tokenSymbol').agg(
        total_buy_cost=('ActualValue', 'sum')
    ).reset_index()

    sell_summary = data[data['trade_type'] == 'sell'].groupby('tokenSymbol').agg(
        total_sell_proceeds=('ActualValue', 'sum')
    ).reset_index()

    summary = pd.merge(buy_summary, sell_summary, on='tokenSymbol', how='outer').fillna(0)

    summary['net_profit'] = summary['total_sell_proceeds'] - summary['total_buy_cost']
    # print(summary)
    return summary



In [10]:
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions, summarize_and_visualize

ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
START_DATE = "2024-12-06 08:00"
END_DATE = "2024-12-12 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE"}
# transfers_df = fetch_and_save_erc20_transfers(
#         address=ADDRESS,
#         start_date=START_DATE,
#         end_date=END_DATE,
#         output_file=OUTPUT_FILE,
#         offset=1000,
#     )
transfers_df = pd.read_csv(OUTPUT_FILE + "/all_erc20_transfers.csv")
# final_record_df = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, BASE_TOKENS)
summary = cal_token_trades(transfers_df, ADDRESS)
print(tabulate(summary, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))



+---------------+------------------+-----------------------+-----------------+
| tokenSymbol   |   total_buy_cost |   total_sell_proceeds |      net_profit |
+===============+==================+=======================+=================+
| APE           |             1.00 |                  0.00 |           -1.00 |
+---------------+------------------+-----------------------+-----------------+
| FLY           |        100000.00 |                  0.00 |      -100000.00 |
+---------------+------------------+-----------------------+-----------------+
| LMWR          |          3734.80 |                  0.00 |        -3734.80 |
+---------------+------------------+-----------------------+-----------------+
| MAGA          |          4747.00 |                  0.00 |        -4747.00 |
+---------------+------------------+-----------------------+-----------------+
| Niza          |          7320.00 |                  0.00 |        -7320.00 |
+---------------+------------------+----------------